In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, year
from pyspark.sql import functions as F

import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
df_sivigila346 = pd.read_csv('../Github/dl-covid19-descriptive-reports/MinSalud/sivigila346/data/ExtraccionEv346.txt',
                             low_memory=False,
                             on_bad_lines='skip',
                             sep='|',
                             usecols = ['PersonaBasicaID',
                                        'Edad',
                                        'FechaHospitalizacion',
                                        'DepartamentoNotificacion',
                                        'FechaNotificacion'])

print('Número filas y columnas: ', df_sivigila346.shape)

# Eliminar registros basura (son 2)
df_sivigila346 = df_sivigila346.dropna(subset=['FechaNotificacion'])
print('Número filas y columnas: ', df_sivigila346.shape)

# Solucionar el formato de una edad, para que la función creada pueda ser utilizada
df_sivigila346['Edad'].mask(df_sivigila346['Edad'] == 'De 75años', 'De 75 años', inplace=True)

# Función para extraer la edad en años
def extraer_edad(edad_str):
    match = re.search(r'\d+', edad_str)
    if not match:
        return None
    edad = int(match.group())
    
    if 'minuto' in edad_str or 'horas' in edad_str or 'día' in edad_str or 'días' in edad_str:
        return 0  # Menos de un año se considera 0 años
    elif 'meses' in edad_str or 'mes' in edad_str:
        return max(edad // 12, 0)  # Convertir meses a años
    return edad  # Edad en años

In [ ]:
# Seleccionar los datos Colombia
df_col = df_sivigila346.copy()

# Eliminar registros con edad "No Definido" (son 37 para Colombia)
df_col = df_col[df_col['Edad'] != 'No Definido']

# Convertir la columna de edad a valores numéricos
df_col['EdadAños'] = df_col['Edad'].apply(extraer_edad)

Segcovid

In [ ]:
spark = SparkSession.builder \
    .master("yarn") \
    .config("spark.executor.instances", "4") \
    .config("spark.executor.cores", "4") \
    .config("spark.executor.memory", "16g") \
    .config("spark.driver.memory", "8g") \
    .config("spark.sql.shuffle.partitions", "200") \
    .appName("Graph2_ICUFR") \
    .getOrCreate()

spark

In [ ]:
def cargar_datos(archivo, columnas):
    try:
        return spark.read.parquet(archivo).select(*columnas)
    except Exception as e:
        print(f"Error al procesar {archivo}: {e}")
        return None

dir_segcovid = {
    "df_2020": "hdfs:///rawdata/segcovid/segcovid_parquet_2020",
    "df_2021": "hdfs:///rawdata/segcovid/segcovid_parquet_2021",
    "df_2022": "hdfs:///rawdata/segcovid/segcovid_parquet_2022",
    "df_2023": "hdfs:///rawdata/segcovid/segcovid_parquet_2023"
}

# Columnas de interés
columnas_segcovid = ["PersonaBasicaID", 
                     "AmbitoAtencion",
                     "FechaIngresoAtencion",
                     "FechaEgresoAtencion",
                     "FechaRegistro",
                     "FechaSeguimiento",
                     "DepartamentoAtencion"]

In [ ]:
# Cargar todos los DataFrames
dfs_segcovid = []
for nombre, archivo in dir_segcovid.items():
    df = cargar_datos(archivo, columnas_segcovid)
    if df is not None:
        dfs_segcovid.append(df)

# Concatenar
if dfs_segcovid:
    df_segcovid = dfs_segcovid[0]
    for df in dfs_segcovid[1:]:
        df_segcovid = df_segcovid.unionByName(df, allowMissingColumns=True)

else:
    df_segcovid = None

# Sólo Bogotá
df_segcovid = df_segcovid.filter(df_segcovid['DepartamentoAtencion'] == '11 - Bogotá D.C.')
df_segcovid = df_segcovid.filter(col("AmbitoAtencion") == "Cuidado Intensivo")

In [ ]:
## Cargar base ID+SDN+FDN

archivo = "hdfs:///analyticsdata/ID_SDN_FDN"
df_age_sex = spark.read.parquet(archivo)

In [ ]:
## Cálculo de edad en SEGCOVID

df_seg_edad = df_segcovid.join(
    df_age_sex,
    df_segcovid["PersonaBasicaID"] == df_age_sex["personaid"],
    how="left"
).select(
    df_segcovid["*"],
    df_age_sex["FDN"]
).persist()

In [ ]:
df_seg_edad = df_seg_edad.withColumn("Edad", year(col("FechaRegistro")) - col("FDN"))

In [ ]:
df_seg_edad = df_seg_edad.where(col("Edad").isNotNull())

In [ ]:
# Estos pasos de acá los incorporo el 07-03-2025 (temporales)
# df_seg_edad = df_seg_edad.drop_duplicates(["PersonaBasicaID","FechaIngresoAtencion"])
# df_seg_edad = df_seg_edad[df_seg_edad['FechaIngresoAtencion'] != '1900-01-01']

In [ ]:
df_seg_edad.select(["PersonaBasicaID", "AmbitoAtencion", "FechaIngresoAtencion",
                    "FechaEgresoAtencion", "FechaRegistro", "FechaSeguimiento","Edad"]).show()

In [ ]:
# Convertir 'FechaRegistro' a tipo date (formato 'AAAA-MM-DD')
df_seg_edad = df_seg_edad.withColumn("FechaRegistro", F.to_date("FechaRegistro", "yyyy-MM-dd"))

In [ ]:
df_seg_edad = df_seg_edad.withColumn( 
    "GrupoEdad",
    F.when((F.col("Edad") >= -1) & (F.col("Edad") <= 8), "0-9")
     .when((F.col("Edad") >= 9) & (F.col("Edad") <= 18), "10-19")
     .when((F.col("Edad") >= 19) & (F.col("Edad") <= 28), "20-29")
     .when((F.col("Edad") >= 29) & (F.col("Edad") <= 38), "30-39")
     .when((F.col("Edad") >= 39) & (F.col("Edad") <= 48), "40-49")
     .when((F.col("Edad") >= 49) & (F.col("Edad") <= 58), "50-59")
     .when((F.col("Edad") >= 59) & (F.col("Edad") <= 68), "60-69")
     .when((F.col("Edad") >= 69) & (F.col("Edad") <= 78), "70-79")
     .otherwise("80+")
)

In [ ]:
waves = { 
    'Wave 1': ('2020-02-26', '2020-09-25'),
    'Wave 2': ('2020-11-01', '2021-03-01'),
    'Wave 3': ('2021-03-01', '2021-09-14'),
    'Wave 4': ('2021-11-20', '2022-03-24')
}

In [ ]:
def wilson_interval(num, tot, z=1.96):
    if tot == 0:
        return 0, 0
    p = num / tot
    denominator = 1 + (z**2) / tot
    center = p + (z**2) / (2*tot)
    margin = z * math.sqrt((p*(1-p)/tot) + (z**2)/(4*(tot**2)))
    lower = (center - margin) / denominator
    upper = (center + margin) / denominator
    return lower, upper

In [ ]:
# Definir etiquetas de grupos de edad según lo definido
labels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80+']
colors = ['#6a5acd', '#66c2a5', '#4daf4a', '#d95f02']
i = 0

plt.figure(figsize=(5, 5))

for wave, (inicio, fin) in waves.items():
    # Convertir fechas a objetos date para usarlos en el filtro
    inicio_date = datetime.strptime(inicio, '%Y-%m-%d').date()
    fin_date = datetime.strptime(fin, '%Y-%m-%d').date()
    
    # -------------------------------------------------------------------------
    # Denominador: total de pacientes en UCI en df_seg_edad, por grupo de edad, dentro de la ola
    df_wave_icu = df_seg_edad.filter(
         (F.col("FechaRegistro") >= F.lit(inicio_date)) &
         (F.col("FechaRegistro") <= F.lit(fin_date))
    )
    
    # Agrupar por GrupoEdad para contar el total de pacientes en UCI
    icu_grouped = df_wave_icu.groupBy("GrupoEdad").count().sort("GrupoEdad")
    
    # Convertir a Pandas para graficar
    grupos_icu = icu_grouped.select("GrupoEdad").rdd.flatMap(lambda x: x).collect()
    dict_icu = {'GrupoEdad': grupos_icu}
    for columna in icu_grouped.columns:
        valores = icu_grouped.select(columna).rdd.flatMap(lambda x: x).collect()
        dict_icu[columna] = valores
    icu_pdf = pd.DataFrame(dict_icu)
    icu_pdf = icu_pdf.set_index("GrupoEdad")["count"].reindex(labels, fill_value=0)
    
    # -------------------------------------------------------------------------
    # Numerador: pacientes en UCI que fallecieron
    # Extraer de los datos en UCI de la ola el identificador y el grupo de edad
    # Se asume que el identificador es "PersonaBasicaID"
    df_wave_icu_ids = df_wave_icu.select("PersonaBasicaID", "GrupoEdad").dropDuplicates()
    zzz = df_wave_icu_ids.select("PersonaBasicaID").rdd.flatMap(lambda x: x).collect()
    dict_icu = {'PersonaBasicaID': zzz}
    for columna in icu_grouped.columns:
        valores = icu_grouped.select(columna).rdd.flatMap(lambda x: x).collect()
        dict_icu[columna] = valores
    
    
    
    icu_ids_pdf = df_wave_icu_ids.toPandas()
    
    # Realizar el merge con df_sivigila, que contiene la información de defunción
    # Se asume que en df_sivigila el identificador se llama "ID"
    merged_deaths = pd.merge(icu_ids_pdf, df_sivigila, left_on="PersonaBasicaID", right_on="ID", how="left")
    
    # Convertir FechaDefuncion a datetime y filtrar los registros que indican defunción (excluyendo "1900-01-01")
    merged_deaths["FechaDefuncion"] = pd.to_datetime(merged_deaths["FechaDefuncion"], errors='coerce')
    deaths_filtered = merged_deaths[merged_deaths["FechaDefuncion"] != pd.Timestamp("1900-01-01")]
    
    # Agrupar por GrupoEdad para contar los fallecidos
    deaths_grouped = deaths_filtered.groupby("GrupoEdad").size().reset_index(name="Deaths")
    deaths_grouped = deaths_grouped.set_index("GrupoEdad").reindex(labels, fill_value=0)["Deaths"]
    
    # -------------------------------------------------------------------------
    # Calcular el porcentaje de riesgo de muerte: (fallecidos / total UCI) * 100
    # Para cada grupo de edad de la ola
    # Se utiliza el intervalo de Wilson para calcular los errores
    risk_percentage = 100 * (deaths_grouped / icu_pdf)
    
    lower_errors = []
    upper_errors = []
    for grupo in labels:
        num = deaths_grouped.get(grupo, 0)
        tot = icu_pdf.get(grupo, 0)
        if tot > 0:
            lower, upper = wilson_interval(num, tot)
            p = num / tot
            lower_errors.append(100 * (p - lower))
            upper_errors.append(100 * (upper - p))
        else:
            lower_errors.append(0)
            upper_errors.append(0)
    
    # Mostrar para verificación en consola (opcional)
    print(f"\nOla: {wave}")
    print("Total en UCI por grupo:")
    print(icu_pdf)
    print("Fallecidos en UCI por grupo:")
    print(deaths_grouped)
    
    # Graficar la línea con sus barras de error
    plt.errorbar(labels, risk_percentage, yerr=[lower_errors, upper_errors],
                 fmt='o-', color=colors[i], ecolor='black', capsize=5,
                 label=wave)
    i += 1

plt.xlabel('Grupo de edad', fontsize=12)
plt.xticks(rotation=45)
plt.ylabel('Porcentaje', fontsize=12)
plt.title('Riesgo de muerte en pacientes en UCI', fontsize=14)
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.gca().yaxis.set_major_formatter(PercentFormatter(decimals=0))
plt.tight_layout()
plt.show()

In [ ]:
from datetime import datetime

# Grafica 
plt.figure(figsize=(5, 5))

labels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80+']
colors = ['#6b6ca3', '#87bcbd', '#6f9954', '#b1615c'] 
i = 0

for wave, (inicio, fin) in waves.items():
    # Convertir fechas a objetos date para usar en los filtros
    inicio_date = datetime.strptime(inicio, '%Y-%m-%d').date()
    fin_date = datetime.strptime(fin, '%Y-%m-%d').date()
    
    # -----------------------------------------------
    # Denominador: total de registros en df_seg_edad por grupo de edad
    df_wave_total = df_seg_edad.filter(
        (F.col("FechaRegistro") >= F.lit(inicio_date)) & (F.col("FechaRegistro") <= F.lit(fin_date))
    )
    
    total_registros_df = df_wave_total.groupBy("GrupoEdad").count()
    
    # Conversión a Pandas para graficar
    grupos_total = total_registros_df.select("GrupoEdad").rdd.flatMap(lambda x: x).collect()
    dict_total = {'GrupoEdad': grupos_total}
    for columna in total_registros_df.columns:
        valores = total_registros_df.select(columna).rdd.flatMap(lambda x: x).collect()
        dict_total[columna] = valores
        
    total_registros_pdf = pd.DataFrame(dict_total)
    total_registros_pdf = total_registros_pdf.set_index("GrupoEdad")["count"].reindex(labels, fill_value=0)
    
    # -----------------------------------------------
    # Numerador: hospitalizados en df_seg_edad por grupo de edad
    # (Si en df_seg_edad existe una columna que indique hospitalización, se podría filtrar aquí, por ejemplo:
    df_wave_icu = df_seg_edad.filter(
             (F.col("FechaRegistro") >= F.lit(inicio_date)) &
             (F.col("FechaRegistro") <= F.lit(fin_date)) &
             (F.col("AmbitoAtencion") == "Cuidado Intensivo")
     )

    df_icu_grouped = df_wave_icu.groupBy("GrupoEdad").count()
    
    grupos_hosp = df_icu_grouped.select("GrupoEdad").rdd.flatMap(lambda x: x).collect()
    dict_hosp = {'GrupoEdad': grupos_hosp}
    for columna in df_icu_grouped.columns:
        valores = df_icu_grouped.select(columna).rdd.flatMap(lambda x: x).collect()
        dict_hosp[columna] = valores
    icu_pdf = pd.DataFrame(dict_hosp)
    icu_pdf = icu_pdf.set_index("GrupoEdad")["count"].reindex(labels, fill_value=0)
    print(icu_pdf)
    
    # Calcular HCR: (hospitalizados / total registros en df_seg_edad)
    ICUCR = icu_pdf / total_registros_pdf
    
    plt.plot(labels, ICUCR, marker='o', linestyle='-', label=wave, color=colors[i])
    i += 1

plt.xlabel('Age Group')
plt.xticks(rotation=90)
plt.ylabel('ICU Case Ratio')
plt.title('Bogotá: ICU CR by Age group Grupo and COVID-19 Waves')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

In [ ]:
from datetime import datetime

# Grafica 
plt.figure(figsize=(5, 5))

labels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80+']
colors = ['#6b6ca3', '#87bcbd', '#6f9954', '#b1615c'] 
i = 0

for wave, (inicio, fin) in waves.items():
    # Convertir fechas a objetos date para usar en los filtros
    inicio_date = datetime.strptime(inicio, '%Y-%m-%d').date()
    fin_date = datetime.strptime(fin, '%Y-%m-%d').date()
    
    # -----------------------------------------------
    # Denominador: total de registros en df_seg_edad por grupo de edad
    df_wave_total = df_seg_edad.filter(
        (F.col("FechaRegistro") >= F.lit(inicio_date)) & (F.col("FechaRegistro") <= F.lit(fin_date))
    )
    
    total_registros_df = df_wave_total.groupBy("GrupoEdad").count()
    
    # Conversión a Pandas para graficar
    grupos_total = total_registros_df.select("GrupoEdad").rdd.flatMap(lambda x: x).collect()
    dict_total = {'GrupoEdad': grupos_total}
    for columna in total_registros_df.columns:
        valores = total_registros_df.select(columna).rdd.flatMap(lambda x: x).collect()
        dict_total[columna] = valores
        
    total_registros_pdf = pd.DataFrame(dict_total)
    total_registros_pdf = total_registros_pdf.set_index("GrupoEdad")["count"].reindex(labels, fill_value=0)
    
    # -----------------------------------------------
    # Numerador: hospitalizados en df_seg_edad por grupo de edad
    # (Si en df_seg_edad existe una columna que indique hospitalización, se podría filtrar aquí, por ejemplo:
    df_wave_icu = df_seg_edad.filter(
             (F.col("FechaRegistro") >= F.lit(inicio_date)) &
             (F.col("FechaRegistro") <= F.lit(fin_date)) &
             (F.col("AmbitoAtencion") == "Cuidado Intensivo")
     )

    df_icu_grouped = df_wave_icu.groupBy("GrupoEdad").count()
    
    grupos_hosp = df_icu_grouped.select("GrupoEdad").rdd.flatMap(lambda x: x).collect()
    dict_hosp = {'GrupoEdad': grupos_hosp}
    for columna in df_icu_grouped.columns:
        valores = df_icu_grouped.select(columna).rdd.flatMap(lambda x: x).collect()
        dict_hosp[columna] = valores
    icu_pdf = pd.DataFrame(dict_hosp)
    icu_pdf = icu_pdf.set_index("GrupoEdad")["count"].reindex(labels, fill_value=0)
    print(icu_pdf)
    
    # Calcular HCR: (hospitalizados / total registros en df_seg_edad)
    ICUCR = icu_pdf / total_registros_pdf
    
    plt.plot(labels, ICUCR, marker='o', linestyle='-', label=wave, color=colors[i])
    i += 1

plt.xlabel('Age Group')
plt.xticks(rotation=90)
plt.ylabel('ICU Case Ratio')
plt.title('Bogotá: ICU CR by Age group Grupo and COVID-19 Waves')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

## Colombia

In [ ]:
# Cargar todos los DataFrames
dfs_segcovid = []
for nombre, archivo in dir_segcovid.items():
    df = cargar_datos(archivo, columnas_segcovid)
    if df is not None:
        dfs_segcovid.append(df)

# Concatenar
if dfs_segcovid:
    df_segcovid = dfs_segcovid[0]
    for df in dfs_segcovid[1:]:
        df_segcovid = df_segcovid.unionByName(df, allowMissingColumns=True)

else:
    df_segcovid = None

# Sólo Bogotá y gente hospitalizada
df_col = df_segcovid

In [ ]:
## Cargar base ID+SDN+FDN

archivo = "hdfs:///analyticsdata/ID_SDN_FDN"
df_age_sex = spark.read.parquet(archivo)

In [ ]:
## Cálculo de edad en SEGCOVID

df_seg_edad = df_col.join(
    df_age_sex,
    df_col["PersonaBasicaID"] == df_age_sex["personaid"],
    how="left"
).select(
    df_col["*"],
    df_age_sex["FDN"]
).persist()

In [ ]:
df_seg_edad = df_seg_edad.withColumn("Edad", year(col("FechaRegistro")) - col("FDN"))

In [ ]:
df_seg_edad = df_seg_edad.where(col("Edad").isNotNull())

In [ ]:
# Convertir 'FechaRegistro' a tipo date (formato 'AAAA-MM-DD')
df_seg_edad = df_seg_edad.withColumn("FechaRegistro", F.to_date("FechaRegistro", "yyyy-MM-dd"))

In [ ]:
df_seg_edad = df_seg_edad.withColumn( 
    "GrupoEdad",
    F.when((F.col("Edad") >= 0) & (F.col("Edad") <= 9), "0-9")
     .when((F.col("Edad") >= 10) & (F.col("Edad") <= 19), "10-19")
     .when((F.col("Edad") >= 20) & (F.col("Edad") <= 29), "20-29")
     .when((F.col("Edad") >= 30) & (F.col("Edad") <= 39), "30-39")
     .when((F.col("Edad") >= 40) & (F.col("Edad") <= 49), "40-49")
     .when((F.col("Edad") >= 50) & (F.col("Edad") <= 59), "50-59")
     .when((F.col("Edad") >= 60) & (F.col("Edad") <= 69), "60-69")
     .when((F.col("Edad") >= 70) & (F.col("Edad") <= 79), "70-79")
     .otherwise("80+")
)

In [ ]:
waves = { 
    'Wave 1': ('2020-02-26', '2020-09-25'),
    'Wave 2': ('2020-11-01', '2021-03-01'),
    'Wave 3': ('2021-03-01', '2021-09-14'),
    'Wave 4': ('2021-11-20', '2022-03-24')
}

In [ ]:
from datetime import datetime

# Grafica 
plt.figure(figsize=(5, 5))

labels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80+']
colors = ['#6b6ca3', '#87bcbd', '#6f9954', '#b1615c'] 
i = 0

for wave, (inicio, fin) in waves.items():
    # Convertir fechas a objetos date para usar en los filtros
    inicio_date = datetime.strptime(inicio, '%Y-%m-%d').date()
    fin_date = datetime.strptime(fin, '%Y-%m-%d').date()
    
    # -----------------------------------------------
    # Denominador: total de registros en df_seg_edad por grupo de edad
    df_wave_total = df_seg_edad.filter(
        (F.col("FechaRegistro") >= F.lit(inicio_date)) & (F.col("FechaRegistro") <= F.lit(fin_date))
    )
    
    total_registros_df = df_wave_total.groupBy("GrupoEdad").count()
    
    # Conversión a Pandas para graficar
    grupos_total = total_registros_df.select("GrupoEdad").rdd.flatMap(lambda x: x).collect()
    dict_total = {'GrupoEdad': grupos_total}
    for columna in total_registros_df.columns:
        valores = total_registros_df.select(columna).rdd.flatMap(lambda x: x).collect()
        dict_total[columna] = valores
        
    total_registros_pdf = pd.DataFrame(dict_total)
    total_registros_pdf = total_registros_pdf.set_index("GrupoEdad")["count"].reindex(labels, fill_value=0)
    
    
    df_wave_icu = df_seg_edad.filter(
             (F.col("FechaRegistro") >= F.lit(inicio_date)) &
             (F.col("FechaRegistro") <= F.lit(fin_date)) &
             (F.col("AmbitoAtencion") == "Cuidado Intensivo")
     )

    df_icu_grouped = df_wave_icu.groupBy("GrupoEdad").count()
    
    grupos_hosp = df_icu_grouped.select("GrupoEdad").rdd.flatMap(lambda x: x).collect()
    dict_hosp = {'GrupoEdad': grupos_hosp}
    for columna in df_icu_grouped.columns:
        valores = df_icu_grouped.select(columna).rdd.flatMap(lambda x: x).collect()
        dict_hosp[columna] = valores
    icu_pdf = pd.DataFrame(dict_hosp)
    icu_pdf = icu_pdf.set_index("GrupoEdad")["count"].reindex(labels, fill_value=0)
    print(icu_pdf)
    print(total_registros_pdf)
    
    # Calcular HCR: (hospitalizados / total registros en df_seg_edad)
    ICUCR = icu_pdf / total_registros_pdf
    
    plt.plot(labels, ICUCR, marker='o', linestyle='-', label=wave, color=colors[i])
    i += 1

plt.xlabel('Age Group')
plt.xticks(rotation=90)
plt.ylabel('ICU Case Ratio')
plt.title('Colombia: ICU CR by Age group Grupo and COVID-19 Waves')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

# Tests

In [ ]:
# df = df.withColumn("GrupoEdad",
#        F.when((F.col("Edad") >= 0) & (F.col("Edad") <= 9), "0-9")
#         .when((F.col("Edad") >= 10) & (F.col("Edad") <= 19), "10-19")
#         .when((F.col("Edad") >= 20) & (F.col("Edad") <= 29), "20-29")
#         .when((F.col("Edad") >= 30) & (F.col("Edad") <= 39), "30-39")
#         .when((F.col("Edad") >= 40) & (F.col("Edad") <= 49), "40-49")
#         .when((F.col("Edad") >= 50) & (F.col("Edad") <= 59), "50-59")
#         .when((F.col("Edad") >= 60) & (F.col("Edad") <= 69), "60-69")
#         .when((F.col("Edad") >= 70) & (F.col("Edad") <= 79), "70-79")
#         .when(F.col("Edad") >= 80, "80+")
#         .otherwise("Desconocido")
# )

# Crear la columna 'GrupoEdad' en df_segcovid usando condiciones
df = df.withColumn(
    "GrupoEdad",
    F.when((F.col("Edad") >= -1) & (F.col("Edad") <= 8), "0-9")
     .when((F.col("Edad") >= 9) & (F.col("Edad") <= 18), "10-19")
     .when((F.col("Edad") >= 19) & (F.col("Edad") <= 28), "20-29")
     .when((F.col("Edad") >= 29) & (F.col("Edad") <= 38), "30-39")
     .when((F.col("Edad") >= 39) & (F.col("Edad") <= 48), "40-49")
     .when((F.col("Edad") >= 49) & (F.col("Edad") <= 58), "50-59")
     .when((F.col("Edad") >= 59) & (F.col("Edad") <= 68), "60-69")
     .when((F.col("Edad") >= 69) & (F.col("Edad") <= 78), "70-79")
     .otherwise("80+")
)

In [ ]:
aggregated_df = df.groupBy("Ola", "GrupoEdad").agg(
    F.count("*").alias("Total_Casos"),
    F.sum(F.when(F.col("AmbitoAtencion") == "Cuidado Intensivo", 1).otherwise(0)).alias("Casos_UCI")
)

In [ ]:
aggregated_df = aggregated_df.withColumn("Tasa_UCI", F.col("Casos_UCI") / F.col("Total_Casos"))

# Mostrar el resultado ordenado por Ola y Grupo de Edad
aggregated_df.orderBy("Ola", "GrupoEdad").show()

In [ ]:
colx = aggregated_df.select('Ola').rdd.flatMap(lambda x: x).collect()
dict = {'Ola': colx}
for columna in aggregated_df.columns:
    colx = aggregated_df.select(columna).rdd.flatMap(lambda x: x).collect()
    dict[columna] = colx

icu_pdf = pd.DataFrame(dict)

In [ ]:
orden_grupos = ["0-9", "10-19", "20-29", "30-39", "40-49", "50-59", "60-69", "70-79", "80+"]

# Definir el color para cada ola
colores = {
    "Ola 1": "red",
    "Ola 2": "blue",
    "Ola 3": "green",
    "Ola 4": "orange"
}

plt.figure(figsize=(10, 6))
# Para cada ola se grafica la tasa usando el código numérico de la categoría para asegurar el orden
for ola in sorted(icu_pdf["Ola"].unique()):
    data_ola = icu_pdf[icu_pdf["Ola"] == ola].sort_values("GrupoEdad")
    # Convertir la categoría a su código numérico para el eje x
    x_vals = data_ola["GrupoEdad"].cat.codes
    plt.plot(x_vals, data_ola["Tasa_UCI"], marker='o', color=colores.get(ola, "black"), label=ola)

# Configurar los ticks del eje x usando los labels ordenados
plt.xticks(ticks=range(len(orden_grupos)), labels=orden_grupos)
plt.xlabel("Grupos de Edad")
plt.ylabel("Tasa UCI")
plt.title("Tasa de UCI por Grupos de Edad y Ola de COVID")
plt.legend(title="Ola")
plt.grid(True)
plt.show()

In [ ]:














df_seg_edad.show(10)



df_seg_edad.select('Edad').distinct().show(200)

df_seg_edad.where(col("Edad").isNull()).distinct().show(200)


# df_seg_edad.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import numpy as np
import re
import seaborn as sns

## Cargar base SIVIGILA346

In [ ]:
df_sivigila346 = pd.read_csv('../Github/dl-covid19-descriptive-reports/MinSalud/sivigila346/data/ExtraccionEv346.txt',
                             low_memory=False,
                             on_bad_lines='skip',
                             sep='|',
                             usecols = ['PersonaBasicaID',
                                        'Edad',
                                        'FechaHospitalizacion',
                                        'DepartamentoNotificacion',
                                        'FechaNotificacion'])

print('Número filas y columnas: ', df_sivigila346.shape)

In [ ]:
# Eliminar registros basura (son 2)
df_sivigila346 = df_sivigila346.dropna(subset=['FechaNotificacion'])
print('Número filas y columnas: ', df_sivigila346.shape)

## Solucionar registro de edad y Función de extracción de edades

In [ ]:
# Solucionar el formato de una edad, para que la función creada pueda ser utilizada
df_sivigila346['Edad'].mask(df_sivigila346['Edad'] == 'De 75años', 'De 75 años', inplace=True)

In [ ]:
# Función para extraer la edad en años
def extraer_edad(edad_str):
    match = re.search(r'\d+', edad_str)
    if not match:
        return None
    edad = int(match.group())
    
    if 'minuto' in edad_str or 'horas' in edad_str or 'día' in edad_str or 'días' in edad_str:
        return 0  # Menos de un año se considera 0 años
    elif 'meses' in edad_str or 'mes' in edad_str:
        return max(edad // 12, 0)  # Convertir meses a años
    return edad  # Edad en años

## Análisis para Bogotá

In [ ]:
# Seleccionar los datos de Bogotá
df_bog = df_sivigila346[df_sivigila346['DepartamentoNotificacion'] == 'Bogotá, D.C.'].copy()

In [ ]:
# Eliminar registros con edad "No Definido" (son 11 para Bogotá)
df_bog = df_bog[df_bog['Edad'] != 'No Definido']

In [ ]:
# Convertir la columna de edad a valores numéricos
df_bog['EdadAños'] = df_bog['Edad'].apply(extraer_edad)

In [ ]:
# Agrupar edades
bins = [0, 9, 19, 29, 39, 49, 59, 69, 79, np.inf]
labels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80+']
df_bog['GrupoEdad'] = pd.cut(df_bog['EdadAños'], bins=bins, labels=labels, right=True)
df_bog.head()

In [ ]:
# Convertir las fechas a formato 'datetime'
df_bog['FechaNotificacion'] = pd.to_datetime(df_bog['FechaNotificacion'], format='%Y%m%d')
df_bog['FechaHospitalizacion'] = pd.to_datetime(df_bog['FechaHospitalizacion'], format='%Y%m%d')

In [ ]:
# Determinar que IDs fueron hospitalizados
fecha_no_hospitalizacion = pd.Timestamp('1900-01-01')
df_bog['Hospitalizado'] = df_bog['FechaHospitalizacion'] != fecha_no_hospitalizacion

In [ ]:
# Descartar los casos donde el ID es igual a 1
df_bog = df_bog[df_bog['PersonaBasicaID'] != '1']

In [ ]:
# Existen 37 registros duplicados. Se mantienen en el resultado
df_bog[df_bog.duplicated(keep=False)].sort_values(by=['PersonaBasicaID'])

In [ ]:
# Definir olas COVID-19 (obtenidas de: https://github.com/TRACE-LAC/covid19-waves-bogota/blob/main/waves/outputs/waves.csv)
waves = {
    'Wave 1': ('2020-02-26', '2020-09-25'),
    'Wave 2': ('2020-11-01', '2021-03-01'),
    'Wave 3': ('2021-03-01', '2021-09-14'),
    'Wave 4': ('2021-11-20', '2022-03-24')
}

In [ ]:
# Grafica 
plt.figure(figsize=(5, 5))
color=['#6b6ca3', '#87bcbd', '#6f9954', '#b1615c']
i = 0
for wave, (inicio, fin) in waves.items():
    df_wave = df_bog[(df_bog['FechaNotificacion'] >= inicio) & (df_bog['FechaNotificacion'] <= fin)]
    
    # Denominador: gente registrada (hospitalizada o no) por Covid por grupo de edad
    total_registros = df_wave.groupby('GrupoEdad').size()
    
    # Numerador: gente que fue hospitalizada por Covid por grupo de edad
    hospitalizados = df_wave[df_wave['Hospitalizado']].groupby('GrupoEdad').size()
    
    # Calcular Hospitalisation Case Ratio (HCR)
    HCR = (hospitalizados / total_registros)
    
    print(hospitalizados)
    
    # Graficar la línea para la ola actual
    plt.plot(labels, HCR, marker='o', linestyle='-', label=wave, color=color[i])
    i = i + 1
    
plt.xlabel('Age group')
plt.xticks(rotation=90)
plt.ylabel('Hospitalisation Case Ratio')
plt.title('Bogotá: HCR by Age group and COVID-19 Waves')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

In [ ]:
# Experimento


## Análisis Nacional

In [ ]:
# Seleccionar los datos Colombia
df_col = df_sivigila346.copy()

# Eliminar registros con edad "No Definido" (son 37 para Colombia)
df_col = df_col[df_col['Edad'] != 'No Definido']

# Convertir la columna de edad a valores numéricos
df_col['EdadAños'] = df_col['Edad'].apply(extraer_edad)

# Agrupar edades
bins = [0, 9, 19, 29, 39, 49, 59, 69, 79, np.inf]
labels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80+']
df_col['GrupoEdad'] = pd.cut(df_col['EdadAños'], bins=bins, labels=labels, right=True)

# Convertir las fechas a formato 'datetime'
df_col['FechaNotificacion'] = pd.to_datetime(df_col['FechaNotificacion'], format='%Y%m%d')
df_col['FechaHospitalizacion'] = pd.to_datetime(df_col['FechaHospitalizacion'], format='%Y%m%d')

# Determinar que IDs fueron hospitalizados
fecha_no_hospitalizacion = pd.Timestamp('1900-01-01')
df_col['Hospitalizado'] = df_col['FechaHospitalizacion'] != fecha_no_hospitalizacion

# Descartar los casos donde el ID es igual a 1
df_col = df_col[df_col['PersonaBasicaID'] != '1']

# Definir olas COVID-19 (obtenidas de: https://github.com/TRACE-LAC/covid19-waves-bogota/blob/main/waves/outputs/waves.csv)
waves = {
    'Wave 1': ('2020-02-26', '2020-09-25'),
    'Wave 2': ('2020-11-01', '2021-03-01'),
    'Wave 3': ('2021-03-01', '2021-09-14'),
    'Wave 4': ('2021-11-20', '2022-03-24')
}

# Grafica 
plt.figure(figsize=(5, 5))
color=['#6b6ca3', '#87bcbd', '#6f9954', '#b1615c']
i = 0
for wave, (inicio, fin) in waves.items():
    df_wave = df_col[(df_col['FechaNotificacion'] >= inicio) & (df_col['FechaNotificacion'] <= fin)]
    
    # Denominador: gente registrada (hospitalizada o no) por Covid por grupo de edad
    total_registros = df_wave.groupby('GrupoEdad').size()
    
    # Numerador: gente que fue hospitalizada por Covid por grupo de edad
    hospitalizados = df_wave[df_wave['Hospitalizado']].groupby('GrupoEdad').size()
    
    # Calcular Hospitalisation Case Ratio (HCR)
    HCR = (hospitalizados / total_registros)
    
    print(hospitalizados)
    
    # Graficar la línea para la ola actual
    plt.plot(labels, HCR, marker='o', linestyle='-', label=wave, color=color[i])
    i = i + 1
    
plt.xlabel('Age group')
plt.xticks(rotation=90)
plt.ylabel('Hospitalisation Case Ratio')
plt.title('Colombia: HCR by Age group and COVID-19 Waves')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

---
---
---

# Cambiando el númerador por SEGCOVID

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, year
from pyspark.sql import functions as F

In [ ]:
spark = SparkSession.builder \
    .master("yarn") \
    .config("spark.executor.instances", "4") \
    .config("spark.executor.cores", "4") \
    .config("spark.executor.memory", "16g") \
    .config("spark.driver.memory", "8g") \
    .config("spark.sql.shuffle.partitions", "200") \
    .appName("Graph2_HCR") \
    .getOrCreate()

spark

In [ ]:
def cargar_datos(archivo, columnas):
    try:
        return spark.read.parquet(archivo).select(*columnas)
    except Exception as e:
        print(f"Error al procesar {archivo}: {e}")
        return None

In [ ]:
dir_segcovid = {
    "df_2020": "hdfs:///rawdata/segcovid/segcovid_parquet_2020",
    "df_2021": "hdfs:///rawdata/segcovid/segcovid_parquet_2021",
    "df_2022": "hdfs:///rawdata/segcovid/segcovid_parquet_2022",
    "df_2023": "hdfs:///rawdata/segcovid/segcovid_parquet_2023"
}

# Columnas de interés
columnas_segcovid = ["PersonaBasicaID", 
                     "AmbitoAtencion",
                     "FechaIngresoAtencion",
                     "DepartamentoAtencion",
                     "FechaRegistro"]

In [ ]:
# Cargar todos los DataFrames
dfs_segcovid = []
for nombre, archivo in dir_segcovid.items():
    df = cargar_datos(archivo, columnas_segcovid)
    if df is not None:
        dfs_segcovid.append(df)

# Concatenar
if dfs_segcovid:
    df_segcovid = dfs_segcovid[0]
    for df in dfs_segcovid[1:]:
        df_segcovid = df_segcovid.unionByName(df, allowMissingColumns=True)

else:
    df_segcovid = None

# Sólo Bogotá y gente hospitalizada
df_segcovid = df_segcovid.filter(df_segcovid['DepartamentoAtencion'] == '11 - Bogotá D.C.')
df_segcovid = df_segcovid.filter(df_segcovid['AmbitoAtencion'] == 'Hospitalización')

## Cargar base ID+SDN+FDN

In [ ]:
archivo = "hdfs:///analyticsdata/ID_SDN_FDN"
df_age_sex = spark.read.parquet(archivo)

## Cálculo de edad en SEGCOVID

In [ ]:
df_seg_edad = df_segcovid.join(
    df_age_sex,
    df_segcovid["PersonaBasicaID"] == df_age_sex["personaid"],
    how="left"
).select(
    df_segcovid["*"],
    df_age_sex["FDN"]
).persist()

In [ ]:
df_seg_edad = df_seg_edad.withColumn("Edad", year(col("FechaRegistro")) - col("FDN"))

In [ ]:
df_seg_edad.show(10)

In [ ]:
df_seg_edad = df_seg_edad.drop_duplicates(["PersonaBasicaID","FechaIngresoAtencion"])

In [ ]:
df_seg_edad.select('Edad').distinct().show(200)

In [ ]:
df_seg_edad.where(col("Edad").isNull()).distinct().show(200)

In [ ]:
df_seg_edad = df_seg_edad.where(col("Edad").isNotNull())
# df_seg_edad.show()

In [ ]:
# ==============================
# Procesamiento para df_bog (denominador)
# ==============================

In [ ]:
# Seleccionar los datos de Bogotá
df_bog = df_sivigila346[df_sivigila346['DepartamentoNotificacion'] == 'Bogotá, D.C.'].copy()

# Eliminar registros con edad "No Definido" (son 11 para Bogotá)
df_bog = df_bog[df_bog['Edad'] != 'No Definido']

# Convertir la columna de edad a valores numéricos
df_bog['EdadAños'] = df_bog['Edad'].apply(extraer_edad)

# Agrupar edades
bins = [0, 9, 19, 29, 39, 49, 59, 69, 79, np.inf]
labels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80+']
df_bog['GrupoEdad'] = pd.cut(df_bog['EdadAños'], bins=bins, labels=labels, right=True)

# Convertir las fechas a formato 'datetime'
df_bog['FechaNotificacion'] = pd.to_datetime(df_bog['FechaNotificacion'], format='%Y%m%d')
df_bog['FechaHospitalizacion'] = pd.to_datetime(df_bog['FechaHospitalizacion'], format='%Y%m%d')

# Determinar que IDs fueron hospitalizados
fecha_no_hospitalizacion = pd.Timestamp('1900-01-01')
df_bog['Hospitalizado'] = df_bog['FechaHospitalizacion'] != fecha_no_hospitalizacion

# Descartar los casos donde el ID es igual a 1
df_bog = df_bog[df_bog['PersonaBasicaID'] != '1']

In [ ]:
# ==============================
# Procesamiento para df_segcovid (numerador)
# ==============================

In [ ]:
# Convertir 'FechaRegistro' a tipo date (formato 'AAAA-MM-DD')
df_seg_edad = df_seg_edad.withColumn("FechaRegistro", F.to_date("FechaRegistro", "yyyy-MM-dd"))

In [ ]:
# Crear la columna 'GrupoEdad' en df_segcovid usando condiciones
df_seg_edad = df_seg_edad.withColumn(
    "GrupoEdad",
    F.when((F.col("Edad") >= -1) & (F.col("Edad") <= 8), "0-9")
     .when((F.col("Edad") >= 9) & (F.col("Edad") <= 18), "10-19")
     .when((F.col("Edad") >= 19) & (F.col("Edad") <= 28), "20-29")
     .when((F.col("Edad") >= 29) & (F.col("Edad") <= 38), "30-39")
     .when((F.col("Edad") >= 39) & (F.col("Edad") <= 48), "40-49")
     .when((F.col("Edad") >= 49) & (F.col("Edad") <= 58), "50-59")
     .when((F.col("Edad") >= 59) & (F.col("Edad") <= 68), "60-69")
     .when((F.col("Edad") >= 69) & (F.col("Edad") <= 78), "70-79")
     .otherwise("80+")
)

# df_seg_edad = df_seg_edad.withColumn(
#     "GrupoEdad",
#     F.when((F.col("Edad") >= 0) & (F.col("Edad") <= 9), "0-9")
#      .when((F.col("Edad") >= 10) & (F.col("Edad") <= 19), "10-19")
#      .when((F.col("Edad") >= 20) & (F.col("Edad") <= 29), "20-29")
#      .when((F.col("Edad") >= 30) & (F.col("Edad") <= 39), "30-39")
#      .when((F.col("Edad") >= 40) & (F.col("Edad") <= 49), "40-49")
#      .when((F.col("Edad") >= 50) & (F.col("Edad") <= 59), "50-59")
#      .when((F.col("Edad") >= 60) & (F.col("Edad") <= 69), "60-69")
#      .when((F.col("Edad") >= 70) & (F.col("Edad") <= 79), "70-79")
#      .otherwise("80+")
# )

In [ ]:
# ==============================
# Definir olas COVID-19 (obtenidas de: https://github.com/TRACE-LAC/covid19-waves-bogota/blob/main/waves/outputs/waves.csv)
waves = {
    'Wave 1': ('2020-02-26', '2020-09-25'),
    'Wave 2': ('2020-11-01', '2021-03-01'),
    'Wave 3': ('2021-03-01', '2021-09-14'),
    'Wave 4': ('2021-11-20', '2022-03-24')
}

In [ ]:
from datetime import datetime
# =============================================================================
# Cálculo del Hospitalisation Case Ratio (HCR) y graficado
# =============================================================================
colors = ['#6b6ca3', '#87bcbd', '#6f9954', '#b1615c']
i = 0

for wave, (inicio, fin) in waves.items():
    # -----------------------------------------------
    # Denominador: casos en df_bog por grupo de edad
    # Filtrar por FechaNotificacion según la ola
    inicio_pd = pd.to_datetime(inicio)
    fin_pd = pd.to_datetime(fin)
    df_wave_bog = df_bog[(df_bog['FechaNotificacion'] >= inicio_pd) & (df_bog['FechaNotificacion'] <= fin_pd)]
    total_registros = df_wave_bog.groupby('GrupoEdad').size().reindex(labels, fill_value=0)
    
    # -----------------------------------------------
    # Numerador: hospitalizados en df_segcovid por grupo de edad
    # Convertir las fechas de inicio y fin a objetos date 
    inicio_date = datetime.strptime(inicio, '%Y-%m-%d').date()
    fin_date = datetime.strptime(fin, '%Y-%m-%d').date()
    
    # Filtrar df_segcovid por FechaRegistro según la ola
    df_wave_seg = df_seg_edad.filter(
        (F.col("FechaRegistro") >= F.lit(inicio_date)) & (F.col("FechaRegistro") <= F.lit(fin_date))
    )
    
    # Agrupar por GrupoEdad y contar
    df_hosp_grouped = df_wave_seg.groupBy("GrupoEdad").count()
    
    # Convertir el resultado a Pandas para graficar
    # AIETQWRHAIFBALJFAJFABÑKJNAÑFJA help!!!
    # El comando toPandas() no funciona
    colx = df_hosp_grouped.select('GrupoEdad').rdd.flatMap(lambda x: x).collect()
    dict = {'GrupoEdad': colx}
    for columna in df_hosp_grouped.columns:
        colx = df_hosp_grouped.select(columna).rdd.flatMap(lambda x: x).collect()
        dict[columna] = colx

    hosp_pdf = pd.DataFrame(dict)
    hosp_pdf = hosp_pdf.set_index("GrupoEdad")["count"].reindex(labels, fill_value=0)
    print(hosp_pdf)
    
    HCR = hosp_pdf / total_registros
    
    plt.plot(labels, HCR, marker='o', linestyle='-', label=wave, color=colors[i])
    i += 1

plt.xlabel('Grupo de Edad')
plt.ylabel('Hospitalisation Case Ratio')
plt.title('Bogotá: HCR por Grupo de Edad y Olas de COVID-19')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

## SIN DUPLICADOS

In [ ]:
from datetime import datetime
# =============================================================================
# Cálculo del Hospitalisation Case Ratio (HCR) y graficado
# =============================================================================
colors = ['#6b6ca3', '#87bcbd', '#6f9954', '#b1615c']
i = 0

for wave, (inicio, fin) in waves.items():
    # -----------------------------------------------
    # Denominador: casos en df_bog por grupo de edad
    # Filtrar por FechaNotificacion según la ola
    inicio_pd = pd.to_datetime(inicio)
    fin_pd = pd.to_datetime(fin)
    df_wave_bog = df_bog[(df_bog['FechaNotificacion'] >= inicio_pd) & (df_bog['FechaNotificacion'] <= fin_pd)]
    total_registros = df_wave_bog.groupby('GrupoEdad').size().reindex(labels, fill_value=0)
    
    # -----------------------------------------------
    # Numerador: hospitalizados en df_segcovid por grupo de edad
    # Convertir las fechas de inicio y fin a objetos date 
    inicio_date = datetime.strptime(inicio, '%Y-%m-%d').date()
    fin_date = datetime.strptime(fin, '%Y-%m-%d').date()
    
    # Filtrar df_segcovid por FechaRegistro según la ola
    df_wave_seg = df_seg_edad.filter(
        (F.col("FechaRegistro") >= F.lit(inicio_date)) & (F.col("FechaRegistro") <= F.lit(fin_date))
    )
    
    # Agrupar por GrupoEdad y contar
    df_hosp_grouped = df_wave_seg.groupBy("GrupoEdad").count()
    
    # Convertir el resultado a Pandas para graficar
    # AIETQWRHAIFBALJFAJFABÑKJNAÑFJA help!!!
    # El comando toPandas() no funciona
    colx = df_hosp_grouped.select('GrupoEdad').rdd.flatMap(lambda x: x).collect()
    dict = {'GrupoEdad': colx}
    for columna in df_hosp_grouped.columns:
        colx = df_hosp_grouped.select(columna).rdd.flatMap(lambda x: x).collect()
        dict[columna] = colx

    hosp_pdf = pd.DataFrame(dict)
    hosp_pdf = hosp_pdf.set_index("GrupoEdad")["count"].reindex(labels, fill_value=0)
    print(hosp_pdf)
    
    HCR = hosp_pdf / total_registros
    
    plt.plot(labels, HCR, marker='o', linestyle='-', label=wave, color=colors[i])
    i += 1

plt.xlabel('Grupo de Edad')
plt.ylabel('Hospitalisation Case Ratio')
plt.title('Bogotá: HCR por Grupo de Edad y Olas de COVID-19')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

---
---

---
---

---
---
---
## Análisis para todos los departamentos

In [ ]:
# Copiar el DataFrame original (para trabajar con todos los departamentos)
df_sivigila346_v2 = df_sivigila346[df_sivigila346['DepartamentoNotificacion'] != 'NO REPORTADO'].copy()

In [ ]:
# https://dgn.isolutions.iso.org/obp/ui#iso:code:3166:CO
df_sivigila346_v2['DepartamentoNotificacion'].replace({'Bogotá, D.C.': 'DC', 
                                                       'Antioquia': 'ANT', 
                                                       'Valle del Cauca': 'VAC', 
                                                       'Cundinamarca': 'CUN', 
                                                       'Santander': 'SAN', 
                                                       'Atlántico': 'ATL', 
                                                       'Boyacá': 'BOY', 
                                                       'Norte de Santander': 'NSA', 
                                                       'Córdoba': 'COR', 
                                                       'Tolima': 'TOL', 
                                                       'Caldas': 'CAL', 
                                                       'Cesar': 'CES', 
                                                       'Meta': 'MET', 
                                                       'Risaralda': 'RIS', 
                                                       'Huila': 'HUI', 
                                                       'Bolívar': 'BOL', 
                                                       'Cauca': 'CAU', 
                                                       'Sucre': 'SUC', 
                                                       'Quindio': 'QUI', 
                                                       'Nariño': 'NAR', 
                                                       'Magdalena': 'MAG', 
                                                       'Casanare': 'CAS', 
                                                       'La Guajira': 'LAG', 
                                                       'Caquetá': 'CAQ', 
                                                       'Putumayo': 'PUT', 
                                                       'Chocó': 'CHO', 
                                                       'Arauca': 'ARA', 
                                                       'Archipiélago de San Andrés, Providencia y Santa Catalina': 'SAP', 
                                                       'Amazonas': 'AMA', 
                                                       'Guaviare': 'GUV', 
                                                       'Guainía': 'GUA', 
                                                       'Vichada': 'VIC', 
                                                       'Vaupés': 'VAU'}, inplace=True)

In [ ]:
# Eliminar registros con edad "No Definido"
df = df_sivigila346_v2[df_sivigila346_v2['Edad'] != 'No Definido'].copy()

In [ ]:
# Convertir la columna de edad a valores numéricos
df['EdadAños'] = df['Edad'].apply(extraer_edad)

In [ ]:
# Agrupar edades
bins = [0, 9, 19, 29, 39, 49, 59, 69, 79, np.inf]
labels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80+']
df['GrupoEdad'] = pd.cut(df['EdadAños'], bins=bins, labels=labels, right=True)

In [ ]:
# Convertir las fechas a formato datetime
df['FechaNotificacion'] = pd.to_datetime(df['FechaNotificacion'], format='%Y%m%d')
df['FechaHospitalizacion'] = pd.to_datetime(df['FechaHospitalizacion'], format='%Y%m%d')

In [ ]:
# Determinar que IDs fueron hospitalizados
fecha_no_hospitalizacion = pd.Timestamp('1900-01-01')
df['Hospitalizado'] = df['FechaHospitalizacion'] != fecha_no_hospitalizacion

In [ ]:
# Descartar los casos donde PersonaBasicaID es igual a '1'
df = df[df['PersonaBasicaID'] != '1']

In [ ]:
waves = {
    'Wave 1': ('2020-02-26', '2020-09-25'),
    'Wave 2': ('2020-11-01', '2021-03-01'),
    'Wave 3': ('2021-03-01', '2021-09-14'),
    'Wave 4': ('2021-11-20', '2022-03-24')
}

In [ ]:
resultados = []

In [ ]:
for wave, (inicio, fin) in waves.items():
    df_wave = df[(df['FechaNotificacion'] >= inicio) & (df['FechaNotificacion'] <= fin)].copy()
    
    # Denominador: gente registrada (hospitalizada o no) por Covid por grupo de edad y departamento
    total = df_wave.groupby(['DepartamentoNotificacion', 'GrupoEdad']).size().reset_index(name='Total')
    
    # Numerador: gente que fue hospitalizada por Covid por grupo de edad y departamento
    hosp = df_wave[df_wave['Hospitalizado']].groupby(['DepartamentoNotificacion', 'GrupoEdad']).size().reset_index(name='Hosp')
    
    # Unir ambos resultados y calcular el HCR
    df_merged = pd.merge(total, hosp, on=['DepartamentoNotificacion', 'GrupoEdad'], how='left')
#     df_merged['Hosp'] = df_merged['Hosp'].fillna(0)
    df_merged['HCR'] = df_merged['Hosp'] / df_merged['Total']
    df_merged['Wave'] = wave
    resultados.append(df_merged)

# DataFrame con los resultados consolidados
df_result = pd.concat(resultados, ignore_index=True)
df_result['HCR'] = df_result['HCR'].fillna(0)

df_result = df_result.sort_values(by=['DepartamentoNotificacion',"GrupoEdad"])

In [ ]:
waves_list = list(waves.keys())

# Grafica
fig, axs = plt.subplots(2, 2, figsize=(14, 20))
axs = axs.flatten()

# Factor de escala burbujas
scale_factor = 1200

for i, wave in enumerate(waves_list):
    df_wave = df_result[df_result['Wave'] == wave]
    scatter = axs[i].scatter(
        x=df_wave['GrupoEdad'].astype(str),
        y=df_wave['DepartamentoNotificacion'],
        s=df_wave['HCR'] * scale_factor,
        alpha=0.6,
        c=df_wave['HCR'],
        cmap='viridis'
    )
    axs[i].set_title(f'Severity: HCR - {wave}')
    axs[i].set_xlabel('Age Group')
    axs[i].set_ylabel('Department')
    
    axs[i].tick_params(axis='x', labelrotation=90)
    
    axs[i].invert_yaxis()
    
    cbar = plt.colorbar(scatter, ax=axs[i])
    cbar.set_label('HCR')

plt.tight_layout()
plt.show()

In [ ]:
# Convertir grupos de edad a posiciones numéricas
group_age_labels = labels  
x_dict = {age: i for i, age in enumerate(group_age_labels)}

# Ordenar los departamentos alfabéticamente
departamentos_sorted = sorted(df_result['DepartamentoNotificacion'].unique())
y_dict = {dep: i for i, dep in enumerate(departamentos_sorted)}

# Definir offsets para cada ola (separación horizontal dentro de cada grupo de edad)
wave_offsets = {
    'Wave 1': -0.3,
    'Wave 2': -0.1,
    'Wave 3': 0.1,
    'Wave 4': 0.3
}

wave_colors = {
    'Wave 1': '#6b6ca3',
    'Wave 2': '#87bcbd',
    'Wave 3': '#6f9954',
    'Wave 4': '#b1615c'
}

plt.figure(figsize=(16, 15))
scale_factor = 1500  # Factor para ajustar el tamaño de las burbujas

for wave in waves.keys():
    df_wave = df_result[df_result['Wave'] == wave]
    # grupo de edad + offset según la ola
    x_vals = df_wave['GrupoEdad'].apply(lambda g: x_dict[str(g)] + wave_offsets[wave])
    y_vals = df_wave['DepartamentoNotificacion'].apply(lambda dep: y_dict[dep])
    sizes = df_wave['HCR'] * scale_factor
    
    plt.scatter(x_vals, y_vals, s=sizes, alpha=0.6, c=wave_colors[wave],
                label=wave, edgecolors='k')

# plt.xticks(ticks=list(x_dict.values()), labels=list(x_dict.keys()), rotation=90)
plt.xticks(ticks=list(x_dict.values()), labels=list(x_dict.keys()))
plt.yticks(ticks=list(y_dict.values()), labels=list(y_dict.keys()))
plt.xlabel('Age Group')
plt.ylabel('Department')
plt.title('HCR By Department, Age Group and Waves')
# plt.legend(title='Waves', bbox_to_anchor=(1.1, 0.5))
plt.grid(True, linestyle='--', alpha=0.5)

plt.gca().invert_yaxis()

legend_handles = [
    Line2D([0], [0], marker='o', color='w', markerfacecolor=wave_colors[wave],
           markersize=10, markeredgecolor='k', label=wave, alpha=0.5)
    for wave in waves.keys()
]
plt.legend(handles=legend_handles, title='Waves', bbox_to_anchor=(1.1, 0.5))

plt.tight_layout()
plt.show()

In [ ]:
# Heatmap
# ------------------------------

waves_list = list(waves.keys())
n_waves = len(waves_list)

fig, axs = plt.subplots(2, 2, figsize=(14, 14))
axs = axs.flatten()

for i, wave in enumerate(waves_list):
    df_wave = df_result[df_result['Wave'] == wave].copy()
    heatmap_data = df_wave.pivot(index='DepartamentoNotificacion', columns='GrupoEdad', values='HCR')
    heatmap_data = heatmap_data.sort_index()  
    sns.heatmap(heatmap_data, ax=axs[i], cmap='viridis', annot=True, fmt=".2f", cbar_kws={'label': 'HCR'})
    axs[i].set_title(f'Severity: HCR  - {wave}')
    axs[i].set_xlabel('Age Group')
    axs[i].set_ylabel('Department')

plt.tight_layout()
plt.show()

# ------------------------------
# Gráfico de Burbujas
# ------------------------------

# fig, axs = plt.subplots(2, 2, figsize=(16, 12))
# axs = axs.flatten()

# scale_factor = 1000

# for i, wave in enumerate(waves_list):
#     df_wave = df_result[df_result['Wave'] == wave].copy()

#     scatter = axs[i].scatter(
#         x=df_wave['GrupoEdad'].astype(str),
#         y=df_wave['DepartamentoNotificacion'],
#         s=df_wave['HCR'] * scale_factor,
#         alpha=0.6,
#         c=df_wave['HCR'],
#         cmap='viridis'
#     )
#     axs[i].set_title(f'HCR Bubble Plot - {wave}')
#     axs[i].set_xlabel('Grupo de Edad')
#     axs[i].set_ylabel('Departamento')
#     # Agregar barra de color en cada subgráfico
#     cbar = plt.colorbar(scatter, ax=axs[i])
#     cbar.set_label('HCR')

# plt.tight_layout()
# plt.show()

# # ------------------------------
# # Gráfico de Barras Agrupadas
# # ------------------------------
# for wave in waves_list:
#     df_wave = df_result[df_result['Wave'] == wave].copy()
#     pivot_df = df_wave.pivot(index='DepartamentoNotificacion', columns='GrupoEdad', values='HCR')
#     pivot_df = pivot_df.sort_index()
    
#     ax = pivot_df.plot(kind='bar', figsize=(18, 6))
#     ax.set_title(f'Grouped Bar Chart of HCR - {wave}')
#     ax.set_xlabel('Departamento')
#     ax.set_ylabel('HCR')
#     plt.xticks(rotation=90)
#     plt.legend(title='Grupo de Edad', bbox_to_anchor=(1.05, 1), loc='upper left')
#     plt.tight_layout()
#     plt.show()


In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(14, 20))
axs = axs.flatten()

# Se define un factor de escala para que el tamaño de las burbujas sea visible
scale_factor = 1000

for i, wave in enumerate(waves_list):
    df_wave = df_result[df_result['Wave'] == wave].sort_values(by=['DepartamentoNotificacion',"GrupoEdad"])
    # Para graficar, se usan las categorías directamente
    scatter = axs[i].scatter(
        x=df_wave['GrupoEdad'].astype(str),
        y=df_wave['DepartamentoNotificacion'],
        s=df_wave['HCR'] * scale_factor,
        alpha=0.6,
        c=df_wave['HCR'],
        cmap='viridis'
    )
    axs[i].set_title(f'HCR Bubble Plot - {wave}')
    axs[i].set_xlabel('Grupo de Edad')
#     axs[i].set_xticklabels(rotation=90)
    axs[i].set_ylabel('Departamento')
    # Agregar barra de color en cada subgráfico
    cbar = plt.colorbar(scatter, ax=axs[i])
    cbar.set_label('HCR')

plt.tight_layout()
plt.show()

In [ ]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


df_clean = df_sivigila346.copy()

# Eliminar registros con edad "No Definido"
df_clean = df_clean[df_clean['Edad'] != 'No Definido']
df_clean['EdadAños'] = df_clean['Edad'].apply(extraer_edad)

# Definir grupos de edad
bins = [0, 9, 19, 29, 39, 49, 59, 69, 79, np.inf]
labels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80+']
df_clean['GrupoEdad'] = pd.cut(df_clean['EdadAños'], bins=bins, labels=labels, right=True)

# Convertir las columnas de fecha a datetime
df_clean['FechaNotificacion'] = pd.to_datetime(df_clean['FechaNotificacion'], format='%Y%m%d')
df_clean['FechaHospitalizacion'] = pd.to_datetime(df_clean['FechaHospitalizacion'], format='%Y%m%d')

# Determinar hospitalización: se considera hospitalizado si la fecha de hospitalización 
# es distinta a '1900-01-01'
fecha_no_hospitalizacion = pd.Timestamp('1900-01-01')
df_clean['Hospitalizado'] = df_clean['FechaHospitalizacion'] != fecha_no_hospitalizacion

# Descartar registros donde PersonaBasicaID es '1'
df_clean = df_clean[df_clean['PersonaBasicaID'] != '1']

# -----------------------------------------------------------------------------
# Definir las olas COVID-19
waves = {
    'Wave 1': ('2020-02-26', '2020-09-25'),
    'Wave 2': ('2020-11-01', '2021-03-01'),
    'Wave 3': ('2021-03-01', '2021-09-14'),
    'Wave 4': ('2021-11-20', '2022-03-24')
}

# =============================================================================
# 1) HEATMAP: Visualiza en cada celda (Departamento x GrupoEdad) el HCR para cada ola
# =============================================================================
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

for i, (wave, (inicio, fin)) in enumerate(waves.items()):
    # Filtrar los datos para la ola actual
    df_wave = df_clean[(df_clean['FechaNotificacion'] >= inicio) & (df_clean['FechaNotificacion'] <= fin)]
    
    # Total de registros por Departamento y Grupo de Edad
    total_df = df_wave.groupby(['DepartamentoNotificacion', 'GrupoEdad']).size().reset_index(name='total')
    # Registros hospitalizados
    hosp_df = df_wave[df_wave['Hospitalizado']].groupby(['DepartamentoNotificacion', 'GrupoEdad']).size().reset_index(name='hosp')
    
    # Combinar y calcular el Hospitalisation Case Ratio (HCR)
    merged_df = pd.merge(total_df, hosp_df, on=['DepartamentoNotificacion', 'GrupoEdad'], how='left')
    merged_df['hosp'] = merged_df['hosp'].fillna(0)
    merged_df['HCR'] = merged_df['hosp'] / merged_df['total']
    
    # Pivot: filas = Departamento, columnas = GrupoEdad
    pivot_df = merged_df.pivot(index='DepartamentoNotificacion', columns='GrupoEdad', values='HCR')
    
    sns.heatmap(pivot_df, annot=True, fmt=".2f", cmap="viridis", ax=axes[i])
    axes[i].set_title(f'{wave}')
    axes[i].set_xlabel('Grupo de Edad')
    axes[i].set_ylabel('Departamento')

plt.tight_layout()
plt.show()

# =============================================================================
# 2) GRÁFICO DE BURBUJAS: Cada burbuja (en Departamento vs Grupo de Edad) tiene tamaño 
# proporcional al HCR para cada ola
# =============================================================================
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

for i, (wave, (inicio, fin)) in enumerate(waves.items()):
    df_wave = df_clean[(df_clean['FechaNotificacion'] >= inicio) & (df_clean['FechaNotificacion'] <= fin)]
    
    total_df = df_wave.groupby(['DepartamentoNotificacion', 'GrupoEdad']).size().reset_index(name='total')
    hosp_df = df_wave[df_wave['Hospitalizado']].groupby(['DepartamentoNotificacion', 'GrupoEdad']).size().reset_index(name='hosp')
    
    merged_df = pd.merge(total_df, hosp_df, on=['DepartamentoNotificacion', 'GrupoEdad'], how='left')
    merged_df['hosp'] = merged_df['hosp'].fillna(0)
    merged_df['HCR'] = merged_df['hosp'] / merged_df['total']
    
    ax = axes[i]
    # Usar scatterplot: x = GrupoEdad, y = Departamento, tamaño = HCR
    sns.scatterplot(data=merged_df, x='GrupoEdad', y='DepartamentoNotificacion', 
                    size='HCR', sizes=(50, 500), alpha=0.7, ax=ax, legend=False)
    ax.set_title(f'{wave}')
    ax.set_xlabel('Grupo de Edad')
    ax.set_ylabel('Departamento')

plt.tight_layout()
plt.show()

# =============================================================================
# 3) GRÁFICO DE BARRAS AGRUPADAS: Cada barra representa el HCR por Grupo de Edad,
# separadas por Departamento para cada ola.
# (Para una opción apilada, se puede cambiar el parámetro "stacked=True")
# =============================================================================
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

for i, (wave, (inicio, fin)) in enumerate(waves.items()):
    df_wave = df_clean[(df_clean['FechaNotificacion'] >= inicio) & (df_clean['FechaNotificacion'] <= fin)]
    
    total_df = df_wave.groupby(['DepartamentoNotificacion', 'GrupoEdad']).size().reset_index(name='total')
    hosp_df = df_wave[df_wave['Hospitalizado']].groupby(['DepartamentoNotificacion', 'GrupoEdad']).size().reset_index(name='hosp')
    
    merged_df = pd.merge(total_df, hosp_df, on=['DepartamentoNotificacion', 'GrupoEdad'], how='left')
    merged_df['hosp'] = merged_df['hosp'].fillna(0)
    merged_df['HCR'] = merged_df['hosp'] / merged_df['total']
    
    # Pivot: filas = GrupoEdad, columnas = Departamento
    pivot_df = merged_df.pivot(index='GrupoEdad', columns='DepartamentoNotificacion', values='HCR')
    pivot_df = pivot_df.reindex(labels)  # Asegurar el orden de los grupos de edad
    
    # Graficar barras agrupadas (usar stacked=True para barras apiladas)
    pivot_df.plot(kind='bar', ax=axes[i], width=0.8, stacked=False)
    axes[i].set_title(f'{wave}')
    axes[i].set_xlabel('Grupo de Edad')
    axes[i].set_ylabel('HCR')
    axes[i].legend(title='Departamento', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

# ------------------------------
# Preprocesamiento y cálculo de HCR (igual que antes)
# ------------------------------

def extraer_edad(edad_str):
    match = re.search(r'\d+', edad_str)
    if not match:
        return None
    edad = int(match.group())
    if 'minuto' in edad_str or 'horas' in edad_str or 'día' in edad_str or 'días' in edad_str:
        return 0
    elif 'meses' in edad_str or 'mes' in edad_str:
        return max(edad // 12, 0)
    return edad

# Trabajar sobre una copia del DataFrame original
df2 = df_sivigila346.copy()

df_sivigila346_v2 = df2[df2['DepartamentoNotificacion'] != 'NO REPORTADO'].copy()

df_sivigila346_v2['DepartamentoNotificacion'].replace({'Bogotá, D.C.': 'DC', 
                                                       'Antioquia': 'ANT', 
                                                       'Valle del Cauca': 'VAC', 
                                                       'Cundinamarca': 'CUN', 
                                                       'Santander': 'SAN', 
                                                       'Atlántico': 'ATL', 
                                                       'Boyacá': 'BOY', 
                                                       'Norte de Santander': 'NSA', 
                                                       'Córdoba': 'COR', 
                                                       'Tolima': 'TOL', 
                                                       'Caldas': 'CAL', 
                                                       'Cesar': 'CES', 
                                                       'Meta': 'MET', 
                                                       'Risaralda': 'RIS', 
                                                       'Huila': 'HUI', 
                                                       'Bolívar': 'BOL', 
                                                       'Cauca': 'CAU', 
                                                       'Sucre': 'SUC', 
                                                       'Quindio': 'QUI', 
                                                       'Nariño': 'NAR', 
                                                       'Magdalena': 'MAG', 
                                                       'Casanare': 'CAS', 
                                                       'La Guajira': 'LAG', 
                                                       'Caquetá': 'CAQ', 
                                                       'Putumayo': 'PUT', 
                                                       'Chocó': 'CHO', 
                                                       'Arauca': 'ARA', 
                                                       'Archipiélago de San Andrés, Providencia y Santa Catalina': 'SAP', 
                                                       'Amazonas': 'AMA', 
                                                       'Guaviare': 'GUV', 
                                                       'Guainía': 'GUA', 
                                                       'Vichada': 'VIC', 
                                                       'Vaupés': 'VAU'}, inplace=True)


df = df_sivigila346_v2[df_sivigila346_v2['Edad'] != 'No Definido']
df['EdadAños'] = df['Edad'].apply(extraer_edad)

bins = [0, 9, 19, 29, 39, 49, 59, 69, 79, np.inf]
labels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80+']
df['GrupoEdad'] = pd.cut(df['EdadAños'], bins=bins, labels=labels, right=True)

df['FechaNotificacion'] = pd.to_datetime(df['FechaNotificacion'], format='%Y%m%d')
df['FechaHospitalizacion'] = pd.to_datetime(df['FechaHospitalizacion'], format='%Y%m%d')

fecha_no_hospitalizacion = pd.Timestamp('1900-01-01')
df['Hospitalizado'] = df['FechaHospitalizacion'] != fecha_no_hospitalizacion

df = df[df['PersonaBasicaID'] != '1']

waves = {
    'Wave 1': ('2020-02-26', '2020-09-25'),
    'Wave 2': ('2020-11-01', '2021-03-01'),
    'Wave 3': ('2021-03-01', '2021-09-14'),
    'Wave 4': ('2021-11-20', '2022-03-24')
}

resultados = []

for wave, (inicio, fin) in waves.items():
    df_wave = df[(df['FechaNotificacion'] >= inicio) & (df['FechaNotificacion'] <= fin)]
    total = df_wave.groupby(['DepartamentoNotificacion', 'GrupoEdad']).size().reset_index(name='Total')
    hosp = df_wave[df_wave['Hospitalizado']].groupby(['DepartamentoNotificacion', 'GrupoEdad']).size().reset_index(name='Hosp')
    df_merged = pd.merge(total, hosp, on=['DepartamentoNotificacion', 'GrupoEdad'], how='left')
    df_merged['Hosp'] = df_merged['Hosp'].fillna(0)
    df_merged['HCR'] = df_merged['Hosp'] / df_merged['Total']
    df_merged['Wave'] = wave
    resultados.append(df_merged)

df_result = pd.concat(resultados, ignore_index=True)
df_result = df_result.sort_values('DepartamentoNotificacion')

# ------------------------------
# Gráfico único de Burbujas combinadas con offset para cada ola
# ------------------------------

# Convertir grupos de edad a posiciones numéricas
group_age_labels = labels  # Usamos la lista de labels definida
x_dict = {age: i for i, age in enumerate(group_age_labels)}

# Ordenar los departamentos alfabéticamente
departamentos_sorted = sorted(df_result['DepartamentoNotificacion'].unique())
y_dict = {dep: i for i, dep in enumerate(departamentos_sorted)}

# Definir offsets para cada ola (separación horizontal dentro de cada grupo de edad)
wave_offsets = {
    'Wave 1': -0.3,
    'Wave 2': -0.1,
    'Wave 3': 0.1,
    'Wave 4': 0.3
}

# Definir colores para cada ola
wave_colors = {
    'Wave 1': '#6b6ca3',
    'Wave 2': '#87bcbd',
    'Wave 3': '#6f9954',
    'Wave 4': '#b1615c'
}

plt.figure(figsize=(16, 15))
scale_factor = 2000  # Factor para ajustar el tamaño de las burbujas

# Iterar por cada ola y graficar en el mismo plot
for wave in waves.keys():
    df_wave = df_result[df_result['Wave'] == wave]
    # Calcular la posición en X (grupo de edad + offset según la ola)
    x_vals = df_wave['GrupoEdad'].apply(lambda g: x_dict[str(g)] + wave_offsets[wave])
    # Posición en Y: Departamento
    y_vals = df_wave['DepartamentoNotificacion'].apply(lambda dep: y_dict[dep])
    sizes = df_wave['HCR'] * scale_factor
    
    plt.scatter(x_vals, y_vals, s=sizes, alpha=0.6, c=wave_colors[wave],
                label=wave, edgecolors='k')

# Configurar los ejes con las etiquetas correspondientes
plt.xticks(ticks=list(x_dict.values()), labels=list(x_dict.keys()))
plt.yticks(ticks=list(y_dict.values()), labels=list(y_dict.keys()))
plt.xlabel('Grupo de Edad')
plt.ylabel('Departamento')
plt.title('Bubble Plot Combinado: HCR por Departamento, Grupo de Edad y Olas')
plt.legend(title='Olas')
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()


### Revisión de duplicados

In [ ]:
df_sivigila346 = pd.read_csv('../Github/dl-covid19-descriptive-reports/MinSalud/sivigila346/data/ExtraccionEv346.txt',
                             low_memory=False,
                             on_bad_lines='skip',
                             sep='|')

print('Número filas y columnas: ', df_sivigila346.shape)

In [ ]:
# Eliminar registros basura (son 2)
df_sivigila346 = df_sivigila346.dropna(subset=['FechaNotificacion'])
print('Número filas y columnas: ', df_sivigila346.shape)

In [ ]:
# Solucionar el formato de una edad, para que la función creada pueda ser utilizada
df_sivigila346['Edad'].mask(df_sivigila346['Edad'] == 'De 75años', 'De 75 años', inplace=True)

# Función para extraer la edad en años
def extraer_edad(edad_str):
    match = re.search(r'\d+', edad_str)
    if not match:
        return None
    edad = int(match.group())
    
    if 'minuto' in edad_str or 'horas' in edad_str or 'día' in edad_str or 'días' in edad_str:
        return 0  # Menos de un año se considera 0 años
    elif 'meses' in edad_str or 'mes' in edad_str:
        return max(edad // 12, 0)  # Convertir meses a años
    return edad  # Edad en años

In [ ]:
# Seleccionar los datos de Bogotá
df_bog = df_sivigila346[df_sivigila346['DepartamentoNotificacion'] == 'Bogotá, D.C.'].copy()
print('Número filas y columnas: ', df_bog.shape)

In [ ]:
# Eliminar registros con edad "No Definido" (son 11 para Bogotá)
df_bog = df_bog[df_bog['Edad'] != 'No Definido']
print('Número filas y columnas: ', df_bog.shape)

In [ ]:
# Convertir la columna de edad a valores numéricos
df_bog['EdadAños'] = df_bog['Edad'].apply(extraer_edad)

In [ ]:
# Agrupar edades
bins = [0, 9, 19, 29, 39, 49, 59, 69, 79, np.inf]
labels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80+']
df_bog['GrupoEdad'] = pd.cut(df_bog['EdadAños'], bins=bins, labels=labels, right=True)

In [ ]:
# Convertir las fechas a formato 'datetime'
df_bog['FechaNotificacion'] = pd.to_datetime(df_bog['FechaNotificacion'], format='%Y%m%d')
df_bog['FechaHospitalizacion'] = pd.to_datetime(df_bog['FechaHospitalizacion'], format='%Y%m%d')

In [ ]:
# Determinar que IDs fueron hospitalizados
fecha_no_hospitalizacion = pd.Timestamp('1900-01-01')
df_bog['Hospitalizado'] = df_bog['FechaHospitalizacion'] != fecha_no_hospitalizacion

In [ ]:
# Descartar los casos donde el ID es igual a 1
df_bog = df_bog[df_bog['PersonaBasicaID'] != '1']

In [ ]:
# Existen 37 registros duplicados. Se mantienen en el resultado
pd.set_option('display.max_columns', None)
df_bog[df_bog['PersonaBasicaID'] == '105532347']

In [ ]:
df_sivigila346[df_sivigila346['PersonaBasicaID'] != '1'][df_sivigila346[df_sivigila346['PersonaBasicaID'] != '1'].duplicated(keep=False)]